In [ ]:
import carla
import random
import time
import numpy as np
import cv2


In [ ]:

# Function to attach the spectator to follow the vehicle
def attach_spectator_to_vehicle(vehicle):
    spectator = world.get_spectator()
    transform = carla.Transform(vehicle.get_transform().transform(carla.Location(x=-6, z=3)), vehicle.get_transform().rotation)
    spectator.set_transform(transform)

# Callback function to process camera images
def camera_callback(image, data_dict):
    array = np.frombuffer(image.raw_data, dtype=np.uint8)
    array = np.reshape(array, (image.height, image.width, 4))
    data_dict['image'] = array[:, :, :3]  # Drop the alpha channel


In [ ]:
# Connect to CARLA Simulator
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)  # Increase timeout for connection stability
world = client.get_world()


In [ ]:
# Reduce quality level for performance
settings = world.get_settings()
settings.synchronous_mode = True  # Enable synchronous mode
settings.fixed_delta_seconds = 0.05  # Fixed time step
world.apply_settings(settings)


In [ ]:
# Get blueprint library and spawn points
bp_lib = world.get_blueprint_library()
spawn_points = world.get_map().get_spawn_points()

In [ ]:
# Spawn a vehicle
vehicle_bp = bp_lib.find("vehicle.lincoln.mkz_2020")
vehicle = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))

if vehicle is None:
    raise RuntimeError("Unable to spawn the vehicle. Please retry.")

# Attach spectator to follow the vehicle
attach_spectator_to_vehicle(vehicle)

In [ ]:
# Camera types to be added
camera_types = [
    "sensor.camera.rgb",
    "sensor.camera.depth",
    # "sensor.camera.semantic_segmentation",
    # "sensor.camera.dvs",
    # "sensor.camera.instance_segmentation",
    # "sensor.camera.optical_flow",
]

# Initialize cameras
cameras = []
camera_data = {}
camera_resolution = (160, 90)  # Reduced resolution for better performance

In [ ]:
for camera_type in camera_types:
    camera_bp = bp_lib.find(camera_type)
    camera_bp.set_attribute("image_size_x", str(camera_resolution[0]))
    camera_bp.set_attribute("image_size_y", str(camera_resolution[1]))
    # camera_bp.set_attribute("fov", "90")  # Field of view
    camera_bp.set_attribute("sensor_tick", "0.10")   # Capture one frame every 0.5 seconds
    
    
    
    # Spawn camera
    camera_transform = carla.Transform(carla.Location(x=1.5, z=2.4))  # Adjust location as needed
    camera = world.spawn_actor(camera_bp, camera_transform, attach_to=vehicle)
    # cameras.append(camera)

    # Initialize data storage
    camera_data[camera_type] = {'image': np.zeros((camera_resolution[1], camera_resolution[0], 3), dtype=np.uint8)}

    # Start listening to the camera
    camera.listen(lambda image, name=camera_type: camera_callback(image, camera_data[name]))

In [ ]:
# Enable autopilot for the vehicle
vehicle.set_autopilot(True)


In [ ]:
# Display camera feeds
cv2.namedWindow("Camera Feed", cv2.WINDOW_AUTOSIZE)

try:
    while True:
        # Combine and show feeds from all cameras
        combined_image = np.hstack([camera_data[camera_type]['image'] for camera_type in camera_types])
        cv2.imshow("Camera Feed", combined_image)

        # Attach the spectator to ensure it's following the vehicle
        attach_spectator_to_vehicle(vehicle)

        if cv2.waitKey(1) == ord('q'):  # Exit on pressing 'q'
            break
finally:
    # Clean up
    cv2.destroyAllWindows()
    for camera in cameras:
        camera.stop()
        camera.destroy()
    vehicle.destroy()